# financial statements testing grounds


In [1]:
import yfinance as yf
import pandas as pd

In [2]:
import requests
import time


api_key = "TULH49F5XQMU8MQ5"
base_url = "https://www.alphavantage.co/query"

def get_fin_stmt(symbol, function):
    url = base_url + "?function=" + function + "&symbol=" + symbol + "&apikey=" + api_key
    response = requests.get(url)
    time.sleep(16)
    if response.status_code != 200:
        print("Error, cannot retrieve" + function + "for " + symbol)
        return None
    data = response.json()
    if "annualReports" not in data:
        print("Error: No " + function + " data found for " + symbol)
        return None
#     print(data)
    statement_data = data["annualReports"]
    return_dct = {}
    
    return statement_data
statement_dct = {}


# retrieving data using above function in compressed format
statements = {}
stocks = ["ENPH", "IBM"]

statement_types =  ["INCOME_STATEMENT", "BALANCE_SHEET", "CASH_FLOW","EARNINGS"]
for stock in stocks:
    statements[stock] = {}
    for statement_type in statement_types:
        statement_key = statement_type.split("_")[0]
        statements[stock][statement_key] = {}
        try:
            fin_statement = get_fin_stmt(stock, statement_type)
            for annual_data in fin_statement:
                date = annual_data["fiscalDateEnding"]

                statements[stock][statement_key][date] = annual_data
        except:
            print("Error retrieving " + statement_type + " for " + stock)


Error: No EARNINGS data found for ENPH
Error retrieving EARNINGS for ENPH
Error: No EARNINGS data found for IBM
Error retrieving EARNINGS for IBM


In [14]:
plot_df = pd.DataFrame(statements["ENPH"]["INCOME"]).T
plot_df

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
2022-12-31,2022-12-31,USD,974595000,2330853000,1356258000,1356258000,448261000,355104000,168846000,526334000,...,27700000,24696000,452048000,54686000,9438000,397362000,388500000,461486000,486182000,397362000
2021-12-31,2021-12-31,USD,554422000,1382049000,827627000,827627000,215832000,233064000,105526000,338590000,...,16700000,9500000,120928000,-24521000,101649000,145449000,142995000,166080000,175580000,145449000
2020-12-31,2020-12-31,USD,345981000,774425000,428444000,428444000,186439000,103621000,55921000,159542000,...,9700000,5092000,119410000,-14585000,21001000,133995000,135352000,140411000,145503000,133995000
2019-12-31,2019-12-31,USD,221245000,624333000,403088000,403088000,102729000,75536000,40381000,118516000,...,7300000,4727000,90114000,-71034000,17843000,161148000,159483000,99805000,104532000,161148000
2018-12-31,2018-12-31,USD,94445000,316159000,221714000,221714000,1596000,56133000,32587000,92849000,...,8300000,1400000,-10229000,1398000,9635000,-11627000,-10229000,464000,1864000,-11627000


In [4]:
plot_df.keys()

Index(['fiscalDateEnding', 'reportedCurrency', 'totalAssets',
       'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue',
       'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables',
       'totalNonCurrentAssets', 'propertyPlantEquipment',
       'accumulatedDepreciationAmortizationPPE', 'intangibleAssets',
       'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments',
       'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets',
       'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'currentAccountsPayable', 'deferredRevenue', 'currentDebt',
       'shortTermDebt', 'totalNonCurrentLiabilities',
       'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt',
       'longTermDebtNoncurrent', 'shortLongTermDebtTotal',
       'otherCurrentLiabilities', 'otherNonCurrentLiabilities',
       'totalShareholderEquity', 'treasuryStock', 'retainedEarnings',
       'commonStock', 'commonStockSharesOutstanding'

In [5]:
def calc_ROIC(EBIT, Taxes, Debt, Equity, Cash_Equiv):
    NOPAT = EBIT - Taxes
    Invested_Capital = Debt + Equity - Cash_Equiv
    ROIC = NOPAT / Invested_Capital
    return ROIC
    

In [6]:
#calcula
#calc_ROIC(statements["ENPH"]["INCOME"]["2022-12-31"]["ebit"], )
int(statements["ENPH"]["INCOME"]["2022-12-31"]["ebit"]) - int(statements[
    "ENPH"]["INCOME"]["2022-12-31"]["incomeTaxExpense"])

406800000

In [7]:
import datetime

In [8]:
#for date in statements["ENPH"]["BALANCE"]:
#    format = "%Y-%m-%d"
    
    
#    print(statements["ENPH"]["BALANCE"][date])

In [9]:
for stock in statements:
      for date in statements[stock]["BALANCE"]:
        print(stock, date, statements[stock]["BALANCE"][date]["commonStockSharesOutstanding"])
    

ENPH 2022-12-31 136441000
ENPH 2021-12-31 133894000
ENPH 2020-12-31 128962000
ENPH 2019-12-31 123109000
ENPH 2018-12-31 107035000
IBM 2022-12-31 906091977
IBM 2021-12-31 898068600
IBM 2020-12-31 892653424
IBM 2019-12-31 887110455
IBM 2018-12-31 892479411


In [10]:
import matplotlib.pyplot as plt


In [11]:
# import os
def plot_lines(df,
              linewidth = 5,
              figsize = (40,20),
              secondary_y = None,
              legend = True,
              pp = None,
              save_fig = False,
              global_fontsize = 50):
    # update global fontsizes
    
    
    plt.rcParams.update({"font.size":global_fontsize})
    fig, ax = plt.subplots(figsize = figsize)
    df.dropna(axis = 0, how = "all").plot.line(
        linewidth = linewidth,
        ax = ax,
        secondary_y = secondary_y,
        legend = legend)
    # remove label "DATE"
    ax.set_xlabel("")
    # rotate x axis labels
    ax.tick_params(axis = "x", rotation = 90)
    # remove ticklines on x and y axis
    ax.tick_params(length=0, which="both")
    # round axis tick values
    # 1. access values
    y_vals = ax.get_yticks()
    ax.set_yticklabels([round(y,2) for y in y_vals])
    
    return fig, ax

In [12]:
#data_dct = {}
#for json_dct in ENPH_IS:
#     date = json_dct["fiscalDateEnding"]
#    data_dct[date] = json_dct
#
#pd.DataFrame(data_dct).T

In [13]:
data

NameError: name 'data' is not defined

In [ ]:
type(data)

import requests

# API key for Alpha Vantage
api_key = 'YOUR_API_KEY'

# Base URL for Alpha Vantage API
base_url = 'https://www.alphavantage.co/query'

# Ticker symbol for AAPL
symbol = 'AAPL'

# Initialize the data dictionary
data = {}

# Function to retrieve financial statement data for a given symbol and statement type
def get_financial_statement(symbol, statement_type):
    # Build the URL for the API call
    url = base_url + '?function=' + statement_type + '&symbol=' + symbol + '&apikey=' + api_key

    # Make the API call
    response = requests.get(url)

    # Check if the API call was successful
    if response.status_code != 200:
        print("Error: Unable to retrieve " + statement_type + " for " + symbol)
        return None

    # Extract the financial statement data from the API response
    data = response.json()
    statement_data = data['annualReports']

    return statement_data

# Retrieve and store the income statement for AAPL
income_statement = get_financial_statement(symbol, 'INCOME_STATEMENT')
data[symbol] = {'Income Statement': income_statement}

# Retrieve and store the balance sheet for AAPL
balance_sheet = get_financial_statement(symbol, 'BALANCE_SHEET')
data[symbol]['Balance Sheet'] = balance_sheet

# Retrieve and store the cash flow statement for AAPL
cash_flow = get_financial_statement(symbol, 'CASH_FLOW')
data[symbol]['Cash Flow'] = cash_flow

# Print the retrieved data
print(data[symbol]['Income Statement'])
print(data[symbol]['Balance Sheet'])
print(data[symbol]['Cash Flow'])
